In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision torchtext spacy
import torch

In [0]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 

import sys
sys.path.append('../')

%matplotlib inline

In [0]:
from torch.autograd import Variable as V
import torchtext
from torchtext import data
import spacy
import numpy as np
from tqdm import tqdm 
 

from torchtext.datasets import WikiText2

In [0]:

TEXT = data.Field(lower=True, tokenize=lambda x: list(x))

device = 'cuda'

In [0]:
train, valid, test = WikiText2.splits(TEXT)

In [0]:
#TEXT.build_vocab(train, vectors="glove.6B.100d")
TEXT.build_vocab(train)

In [0]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100
eval_batch_size = 128

In [0]:
train_loader, val_loader, test_loader = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=batch_size,
    bptt_len=sequence_length,
    device=device,
    repeat=False)

In [0]:
batch, (torchtext_data) = next(enumerate(train_loader))

In [0]:
data = torchtext_data.text

In [0]:
targets = torchtext_data.target

In [0]:
data[:5, :3]

tensor([[ 2,  9, 16],
        [30, 22, 22],
        [ 2,  2,  4],
        [33, 20, 11],
        [ 2,  5, 10]], device='cuda:0')

In [0]:
targets.reshape(30, 128)[:5, :3]

tensor([[30, 22, 22],
        [ 2,  2,  4],
        [33, 20, 11],
        [ 2,  5, 10],
        [26, 10,  2]], device='cuda:0')

In [0]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [0]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab)
    hidden = model.init_hidden(eval_batch_size)
    for i, (torchtext_data) in enumerate(data_loader):
        data, targets = torchtext_data.text, torchtext_data.target.view(-1)
        data = data.to(device)
        targets = targets.to(device)
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / len(data_loader)

In [0]:
def train():
    model.train()
    total_loss = 0
#     for batch in tqdm(train_loader):
#     # reset the hidden state or else the model will try to backpropagate to the
#     # beginning of the dataset, requiring lots of time and a lot of memory
    #model.reset_history()
    ntokens = len(TEXT.vocab)
    for batch, (torchtext_data) in enumerate(train_loader):
        data, targets = torchtext_data.text, torchtext_data.target.view(-1)
        data = data.to(device)
        targets = targets.to(device)
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [0]:
len(TEXT.vocab)

245

In [0]:

ntokens = len(TEXT.vocab)
ninp = 128
#model = RNNModel('LSTM', ntokens, ninp, 128, 2, batch_size, 0.3)
model = RNNModel('LSTM', ntokens, 128, 128, 3, 0.3).to(device)

#model.encoder.weight.data.copy_(weight_matrix)
criterion = nn.CrossEntropyLoss()

In [0]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long().cuda()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    return ''.join(out)

In [0]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 30):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')

sample:
 eصc=dem iiié7r=cef eieṯtemiuincu uuī etom,tcžloxai 

| epoch   1 |   100/ 2808 batches | lr 4.00 | loss  3.13 | ppl    22.89
| epoch   1 |   200/ 2808 batches | lr 4.00 | loss  3.09 | ppl    21.96
| epoch   1 |   300/ 2808 batches | lr 4.00 | loss  3.09 | ppl    21.93
| epoch   1 |   400/ 2808 batches | lr 4.00 | loss  3.08 | ppl    21.74
| epoch   1 |   500/ 2808 batches | lr 4.00 | loss  3.09 | ppl    21.92
| epoch   1 |   600/ 2808 batches | lr 4.00 | loss  3.08 | ppl    21.85
| epoch   1 |   700/ 2808 batches | lr 4.00 | loss  3.09 | ppl    21.88
| epoch   1 |   800/ 2808 batches | lr 4.00 | loss  3.08 | ppl    21.74
| epoch   1 |   900/ 2808 batches | lr 4.00 | loss  3.09 | ppl    21.90
| epoch   1 |  1000/ 2808 batches | lr 4.00 | loss  3.08 | ppl    21.84
| epoch   1 |  1100/ 2808 batches | lr 4.00 | loss  3.01 | ppl    20.37
| epoch   1 |  1200/ 2808 batches | lr 4.00 | loss  2.90 | ppl    18.25
| epoch   1 |  1300/ 2808 batches | lr 4.00 | loss  2.83 | ppl    16.97
| 

KeyboardInterrupt: ignored

In [0]:
print('sample:\n', generate_(50), '\n')

sample:
  thit dab frane the rixings apranto , tros she ter 



's'